# KoBERT 사용하기(HuggingFace Library)
- KoBERT(SKT)도 있고 KorBERT(ETRI)도 있고 KoBART, KoGPT도 있는 것 같다. 어떤 걸 써야 하지?! 
- ETRI를 활용한 KorBertSum 
  - https://velog.io/@raqoon886/KorBertSum-SummaryBot 
  - https://github.com/raqoon886/KorBertSum
- KoBART를 이용한 요약
  - http://blog.ju-ing.co.kr/posts/KoBART-summarization/ 
  - https://younghwani.github.io/posts/kobart-summary-3/ 

Direct Access를 위한 data preprocess functions 정의하기

In [1]:
import re

def del_bracket(s):
  pattern = r'\([^)]*\)'  # ()
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\[[^)]*\]'  # []
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\<[^)]*\>'  # <>
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\{[^)]*\}'  # {}
  s = re.sub(pattern=pattern, repl='', string=s)

  return s

def del_special_num(s):
  pattern = r'[^a-zA-Z가-힣]'
  s = re.sub(pattern=pattern, repl=' ', string=s)

  return s

def del_unit(s):
  units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
  for unit in units:
    s = s.lower() # 대문자를 소문자로 변환
    s = s.replace(unit, '')
  return s

def del_whitespace(s):
  return " ".join(s.split())

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
def del_stopwords(s):
  stopwords = open("data/stopwords.txt", 'r', encoding="utf-8").read().split()
  #print(stopwords)
  s_o=s.split()
  s_f=[]
  for w in s_o:
    if w.strip() not in stopwords:
      s_f.append(w.strip())
  return " ".join(s_f)

In [2]:
#modelname = "beomi/KcELECTRA-base"
modelname = "klue/bert-base"
#modelname = "beomi/kcbert-base"
#modelname = "monologg/kobert"

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_cosine_schedule_with_warmup, BertTokenizer, BertModel

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import pandas as pd

train=pd.read_excel('data/Training.xlsx')
train=pd.DataFrame({'s1': train['감정_대분류'], 's2': train['감정_소분류'], 't1': train['사람문장1']})
train.head()

val=pd.read_excel('data/Validation.xlsx')
val=pd.DataFrame({'s1': val['감정_대분류'], 's2': val['감정_소분류'], 't1': val['사람문장1']})

In [5]:
#기쁨 불안 당황 슬픔 분노 상처 
s12label={'기쁨':0, '불안': 1, '당황':2, '슬픔': 3, '분노': 4, '상처':5}
train_l=[]
for t, s in zip(train['t1'], train['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s.strip()]])

for t, s in zip(val['t1'], val['s1']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_l.append([t, s12label[s]])

print(train_l[:10])

[['일은 끝이 없을까 화가 난다', 4], ['달에 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가', 4], ['회사에 신입이 들어왔는데 말투가 거슬려 애를 매일 봐야 한다고 생각하니까 스트레스 받아', 4], ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 데 정말 분하고 섭섭해', 4], ['전 입사한 신입사원이 나를 무시하는 같아서 너무 화가', 4], ['직장에 다니고 있지만 시간만 버리는 거 같아 진지하게 진로에 대한 고민이 생겨', 4], ['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔 나도 섭섭해', 4], ['퇴사한 지 됐지만 천천히 직장을 구해보려고', 0], ['졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어', 1], ['요즘 직장생활이 너무 편하고 좋은 같아', 0]]


In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        self.sentences = [tokenizer.encode_plus(i[sent_idx], add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, idx):
        token_ids = torch.tensor(self.sentences[idx]['input_ids'], dtype=torch.long)
        valid_length = torch.tensor(len(self.sentences[idx]['input_ids']), dtype=torch.long)
        segment_ids = torch.tensor(self.sentences[idx]['token_type_ids'], dtype=torch.long)
        attention_mask = torch.tensor(self.sentences[idx]['attention_mask'], dtype=torch.long)
        label = torch.tensor(self.labels[idx])
        return token_ids, valid_length, segment_ids, attention_mask, label

    def __len__(self):
        return (len(self.labels))

In [7]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 10
learning_rate =  1e-5

In [8]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# Preparing the dataset
dataset_train, dataset_test = train_test_split(train_l, test_size=0.2, random_state=0)
print(len(dataset_train), len(dataset_test))

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Ensure all data is a string
dataset_train = [[str(i[0]), i[1]] for i in dataset_train]
dataset_test = [[str(i[0]), i[1]] for i in dataset_test]

# Creating instances of the BERTDataset class for train and test sets
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, max_len)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len)


46616 11655


c:\Users\Chiwoong\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chiwoong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
data_train[0]

(tensor([   2, 4229, 3686, 6509, 4819, 2069, 1085, 6205, 2097, 8402,  568,  555,
         2227,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 tensor(64),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(3, dtype=torch.int32))

In [10]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [11]:
num_classes = 6

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [12]:
# Code for checking bias

from collections import Counter
labels = []
for _, (_, _, _, _, label) in enumerate(tqdm(test_dataloader)):
    labels.extend(label.numpy())
print(Counter(labels))

100%|██████████| 183/183 [00:00<00:00, 530.38it/s]

Counter({4: 2080, 5: 2039, 1: 2036, 3: 2032, 2: 2001, 0: 1467})


In [13]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.7).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(out, label):
    predicted = out.argmax(1)
    correct = (predicted == label).sum().item()
    total = label.size(0)
    accuracy = correct / total
    # print(f'Predicted: {predicted}, Label: {label}, Correct: {correct}, Total: {total}, Accuracy: {accuracy}')
    return accuracy

    
train_dataloader

c:\Users\Chiwoong\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
num_augmentations = 0
len_data = len(dataset_train) + len(dataset_test)

In [15]:
# For time-based tensorboard store

from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d")
log_dir = f'./logs/{timestamp}-model{modelname}-data{len_data}/a{num_augmentations}-class{num_classes}-batch{batch_size}-lr{learning_rate}-epoch{num_epochs}-maxlen{max_len}'

In [16]:
from torch.utils.tensorboard import SummaryWriter

In [17]:
from tqdm.notebook import tqdm

best_acc = 0
max_patience = 5
patience = 0
PATH = log_dir
writer = SummaryWriter(PATH)

# print("Starting training loop...")
for e in range(num_epochs):
    # print("Starting epoch ", e+1)
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
    for batch_id, (token_ids, valid_length, segment_ids, attention_mask, label) in enumerate(tqdm(train_dataloader)):
        # print("Starting batch ", batch_id+1)    
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        attention_mask = attention_mask.to(device)
        out = model(token_ids, valid_length, segment_ids, attention_mask)
        # print(f'Output: {out.argmax(1)}, Label: {label}')

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            writer.add_scalar('loss/train_loss', loss.data.cpu().numpy(), e*len(train_dataloader)+batch_id+1)
            writer.add_scalar('acc/train_acc', train_acc / (batch_id+1), e*len(train_dataloader)+batch_id+1)
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, attention_mask, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        attention_mask = attention_mask.to(device)
        out = model(token_ids, valid_length, segment_ids, attention_mask)
        # print(f'Output: {out.argmax(1)}, Label: {label}')
        
        test_acc += calc_accuracy(out, label)
        loss = loss_fn(out, label)
    writer.add_scalar('acc/test_acc', test_acc / (batch_id+1), e+1)        
    writer.add_scalar('loss/test_loss', loss.data.cpu().numpy(), e+1)        
    print("epoch {} loss {} test acc {}".format(e+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    if test_acc/(batch_id+1)>best_acc:
      best_acc=test_acc/(batch_id+1)
      patience=0
      torch.save(model, PATH + '{}_{}_model.pt'.format(e+1, test_acc/(batch_id+1)))
    else:
      patience+=1
    if patience>max_patience:
      break
    test_acc = 0  # Reset test accuracy for the next epoch

  0%|          | 0/729 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7601990699768066 train acc 0.25
epoch 1 batch id 11 loss 1.8913229703903198 train acc 0.22585227272727273
epoch 1 batch id 21 loss 1.8737117052078247 train acc 0.19419642857142858
epoch 1 batch id 31 loss 1.7930320501327515 train acc 0.19405241935483872
epoch 1 batch id 41 loss 1.880544900894165 train acc 0.19397865853658536
epoch 1 batch id 51 loss 2.030775308609009 train acc 0.19148284313725492
epoch 1 batch id 61 loss 1.7764157056808472 train acc 0.18596311475409835
epoch 1 batch id 71 loss 1.840705394744873 train acc 0.18485915492957747
epoch 1 batch id 81 loss 1.9575343132019043 train acc 0.18113425925925927
epoch 1 batch id 91 loss 1.9175658226013184 train acc 0.18097527472527472
epoch 1 batch id 101 loss 1.973516821861267 train acc 0.1806930693069307
epoch 1 batch id 111 loss 1.8978997468948364 train acc 0.18074324324324326
epoch 1 batch id 121 loss 1.8490214347839355 train acc 0.18130165289256198
epoch 1 batch id 131 loss 1.8571929931640625 train acc 0

  0%|          | 0/183 [00:00<?, ?it/s]

epoch 1 loss 1.4514329433441162 test acc 0.4323160616705699


  0%|          | 0/729 [00:00<?, ?it/s]

...

epoch 14 loss 1.8406202793121338 test acc 0.5825161007025761


  0%|          | 0/729 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.4947088658809662 train acc 0.828125
epoch 15 batch id 11 loss 0.3633843660354614 train acc 0.8792613636363636
epoch 15 batch id 21 loss 0.18833790719509125 train acc 0.8712797619047619
epoch 15 batch id 31 loss 0.4055154323577881 train acc 0.8679435483870968
epoch 15 batch id 41 loss 0.3900715708732605 train acc 0.8666158536585366
epoch 15 batch id 51 loss 0.17107315361499786 train acc 0.8648897058823529
epoch 15 batch id 61 loss 0.5292547941207886 train acc 0.8632172131147541
epoch 15 batch id 71 loss 0.5735335946083069 train acc 0.8637764084507042
epoch 15 batch id 81 loss 0.4539036750793457 train acc 0.8630401234567902
epoch 15 batch id 91 loss 0.40511178970336914 train acc 0.8607486263736264
epoch 15 batch id 101 loss 0.8490390777587891 train acc 0.8575185643564357
epoch 15 batch id 111 loss 0.5160422325134277 train acc 0.8579673423423423
epoch 15 batch id 121 loss 0.21110466122627258 train acc 0.8575671487603306
epoch 15 batch id 131 loss 0.3324091136455

  0%|          | 0/183 [00:00<?, ?it/s]

epoch 15 loss 2.110285520553589 test acc 0.5790032201405153


To load the model from PATH?

In [16]:
# Specify the path to your saved model file
model_path = '' # need model path dir: PATH + '{}_{}_model.pt

# Load the model
model = torch.load(model_path)

# Make sure to call model.eval() if you're loading the model for evaluation
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: ''

# Interactive Test

In [17]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
PATH="/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/KoBERT-topic-dr+0.7-1122/2_0.6723177083333334_model_t.pt"
#model = BERTClassifier2(bertmodel,  dr_rate=0.7).to(device)
model=torch.load(PATH)
#{'연애, 결혼, 출산': 0, '건강, 죽음': 1, '학업 및 진로': 2, '재정': 3, '재정, 은퇴, 노후준비': 4, 
#'건강': 5, '학교폭력/따돌림': 6, '대인관계': 7, '진로, 취업, 직장': 8, '가족관계': 9, '직장, 업무 스트레스': 10, 
#'대인관계(부부, 자녀)': 11}
s12label={'기쁨':0, '불안': 1, '당황':2, '슬픔': 3, '분노': 4, '상처':5}
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            
            if np.argmax(logits) == 0:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 1:
                test_eval.append("건강, 죽음")
            elif np.argmax(logits) == 2:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 3:
                test_eval.append("재정")
            elif np.argmax(logits) == 4:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 5:
                test_eval.append("건강")

            elif np.argmax(logits) == 6:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 7:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 8:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 9:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 10:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 11:
                test_eval.append("대인관계(부부, 자녀)")

        print(">> 입력하신 내용이 " + test_eval[0] + "에 대한 발화로 추정됩니다.")

NameError: name 'get_tokenizer' is not defined

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence.strip() == '0' :
        break
    predict(sentence)
    print("\n")

# KoBERT 주제 분석(???; 중복된 내용으로 판단됨)

In [ ]:
train_o=pd.read_excel('/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/감성대화말뭉치(최종데이터)_Training.xlsx')
val_o=pd.read_excel('/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/감성대화말뭉치(최종데이터)_Validation.xlsx')
train_t=pd.DataFrame({'topic': train_o['상황키워드'], 't1': train_o['사람문장1']})
train_t.head()

val_t=pd.DataFrame({'topic': val_o['상황키워드'], 't1': val_o['사람문장1']})


In [ ]:
topic2label={}
for idx, topic in enumerate(train_t['topic'].unique()):
  topic2label[topic]=idx

train_tt=[]
for t, s in zip(train_t['t1'], train_t['topic']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_tt.append([t, topic2label[s.strip()]])

for t, s in zip(val_t['t1'], val_t['topic']):
  t=del_bracket(t)
  t=del_special_num(t)
  t=del_whitespace(t)
  t=del_stopwords(t)
  train_tt.append([t, topic2label[s]])

dataset_train_t, dataset_test_t = train_test_split(train_tt, test_size=0.2, random_state=0)

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train_t = BERTDataset(dataset_train_t, 0, 1, tok, max_len, True, False)
data_test_t = BERTDataset(dataset_test_t, 0, 1, tok, max_len, True, False)

train_dataloader_t = torch.utils.data.DataLoader(data_train_t, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader_t = torch.utils.data.DataLoader(data_test_t, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
class BERTClassifier2(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=12,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
print(topic2label)

In [ ]:
#BERT 모델 불러오기
model_t = BERTClassifier(bertmodel,  dr_rate=0.7).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_t.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_t.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader_t) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader_t

In [ ]:
best_acc=0
max_patience=5
patience=0
PATH = '/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/KoBERT-topic-dr+0.7-1122/'
writer = SummaryWriter(PATH)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_t.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader_t)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_t(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_t.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            writer.add_scalar('loss/train_loss', loss.data.cpu().numpy(), e*len(train_dataloader_t)+batch_id+1)
            writer.add_scalar('acc/train_acc', train_acc / (batch_id+1), e*len(train_dataloader_t)+batch_id+1)
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model_t.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader_t)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_t(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        loss = loss_fn(out, label)
    writer.add_scalar('acc/test_acc', test_acc / (batch_id+1), e+1)        
    writer.add_scalar('loss/test_loss', loss.data.cpu().numpy(), e+1)        
    print("epoch {} loss {} test acc {}".format(e+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    
    if test_acc/(batch_id+1)>best_acc:
      best_acc=test_acc/(batch_id+1)
      patience=0
      torch.save(model_t, PATH + '{}_{}_model_t.pt'.format(e+1, test_acc/(batch_id+1)))
    else:
      patience+=1
    if patience>max_patience:
      break
    

# Aihub 감성 데이터

In [ ]:
!cd /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화

In [ ]:
!unzip /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/Training/감성대화말뭉치_원천데이터_Training.zip -d /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화

In [ ]:
!unzip /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/Training/감성대화말뭉치_최종데이터_Training.zip -d /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화

In [ ]:
!unzip /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/Validation/감성대화말뭉치_원천데이터_Validation.zip -d /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화

In [ ]:
!unzip /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/Validation/감성대화말뭉치_최종데이터_Validation.zip -d /content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화

In [ ]:
!pip install xlrd

In [ ]:
import pandas as pd
train=pd.read_excel('/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/감성대화말뭉치(최종데이터)_Training.xlsx')
train=pd.DataFrame({'s1': train['감정_대분류'], 's2': train['감정_소분류'], 't1': train['사람문장1']})
train.head()

In [ ]:
val=pd.read_excel('/content/drive/MyDrive/Deepdaiv-SentimentAnalysis/감성대화/감성대화말뭉치(최종데이터)_Validation.xlsx')
val=pd.DataFrame({'s1': val['감정_대분류'], 's2': val['감정_소분류'], 't1': val['사람문장1']})

In [ ]:
for s1 in train['s1'].unique():
  print(s1)
  print(train.loc[train['s1']==s1]['s2'].value_counts())
  print()

#기쁨 불안 당황 슬픔 분노 상처 

In [ ]:
for s1 in val['s1'].unique():
  print(s1)
  print(val.loc[val['s1']==s1]['s2'].value_counts())
  print()